# Das Projekt
Eine große Supermarktkette betreibt 300 Filialen in Deutschland. Als Grundlage für die Personalplanung
muss für jede Filiale entschieden werden, wie viele MitarbeiterInnen an einem bestimmten Tag in der
Filiale sein müssen. Der Bedarf richtet sich dabei nach der erwarteten Anzahl von Kunden bzw. dem
erwarteten Umsatz. Bisher wird der Bedarf individuell von den FilialleiterInnen ermittelt.
In dieser Aufgabe soll anhand von historischen Umsatzzahlen, Filial-, Angebots- und Wettbewerberdaten ein Modell entwickelt werden, das für jede Filiale den erwarteten Umsatz möglichst gut vorhersagt
und damit allen FilialleiterInnen eine passgenaue Personalplanung ermögSpalte Sales zur Datei dmml1_test.csv hinzu.

## 2.1 Aufgaben
Für die Entwicklung des Modells muss der gesamte Machine Learning Workflow durchlaufen werden,
von der Datenvorverarbeitung und -analyse über das Modelltraining verschiedener Modelle bis zur Auswertung der Modellgüte.
Bearbeiten Sie dazu folgende Aufgaben:

### Aufgabe 1 (10 Punkte)
Untersuchen Sie die Daten. Beantworten Sie dabei folgende Fragen bzw. Aufgaben:
1. Wie viele Datenpunkte gibt es pro Store?
2. Visualisieren Sie die Verteilung der Verkaufszahlen für einige zufällig ausgewählte Stores und beschreiben Sie typische Muster, die Sie erkennen.
3. Beschreiben Sie einige prägnante Zusammenhänge, die Ihnen zwischen verschiedenen Features
auffallen.

### Aufgabe 2 (10 Punkte)
Führen Sie geeignete Vorverarbeitungsschritte durch, z.B. Behandlung von Ausreißern und fehlenden
Werten, Skalierung der Features.

### Aufgabe 3 (15 Punkte)
Generieren Sie neue Features, die für die Vorhersage des Umsatzes aussagekräftig sein könnten.

### Aufgabe 4 (30 Punkte)
Trainieren Sie drei verschiedene Modelle, die in der Vorlesung behandelt wurden: ein lineares Modell
(einfache lineare Regression, Ridge, Lasso), einen Entscheidungsbaum und ein Ensemble-Modell (Gradient Boosting oder Random Forest)
1. Optimieren sie Hyperparameter der Modelle mittels Suche und Kreuzvalidierung. Überlegen Sie
dazu zunächst (mit Hilfe der Vorlesungsunterlagen und der Dokumentation der Methoden in
scikit-learn ), was für die jeweiligen Modelle Hyperparameter sind und für welche sich eine
Optimierung ggf. lohnen könnte.
2. Welches sind die wichtigsten Features für die jeweiligen Modelle?

### Aufgabe 5 (15 Punkte)
Das Ziel der Modelle ist es, die FilialleiterInnen bei der Erstellung der Schichtpläne zu unterstützen.
Dazu leiten diese aus der Umsatzvorhersage ihres Modells einen Bedarf an MitarbeiterInnen ab, die an
diesem Tag zur Schicht eingeteilt werden. Wenn die Umsatzvorhersage deutlich höher ist als der tatsächliche Umsatz, so werden zu viele Mitarbeiter in der Filiale sein und es entstehen unnötige Lohnkosten. Nehmen Sie an, dass pro Tag, an dem die Vorhersage den tatsächlichen Umsatz um mehr als 3000
EUR überschätzt 150 EUR Kosten (zusätzliche Lohnkosten) anfallen. Wenn die Vorhersage um mehr
als 6000 EUR höher liegt, betragen die Kosten sogar 250 EUR. Falls die Vorhersage den tatsächlichen
Umsatz unterschätzt, so sind die Kosten schwieriger zu quantifizieren. Darunter fallen Kosten für etwaige Überstunden aber auch Kosten durch Unzufriedenheit der Kunden und der Mitarbeiter (da z.B.
lange Wartezeiten in Kauf genommen werden müssen), die sich langfristig negativ auswirken. Nehmen
Sie vereinfachend an, dass 100 EUR Kosten entstehen, wenn das Modell den Umsatz um mehr als 4000
EUR unterschätzt. Welches ihrer Modelle wird nach diesen Annahmen voraussichtlich die geringsten
Gesamtkosten verursachen?

### Aufgabe 6 (20 Punkte)
Erstellen Sie eine Vorhersage für die Testdaten mit dem besten Modell (bezüglich ihrer Metrik aus der
vorherigen Aufgabe). Geben Sie diese Vorhersage als .csv Datei mit ab. Fügen Sie dazu einfach eine
Spalte Sales zur Datei dmml1_test.csv hinzu.